# [Create Agent with Azure Functions](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview#create-agent-with-azure-function-call)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

The **STORAGE_SERVICE_ENDPOINT** string is used to triggering the Azure function.<br/>

Notes:
- Inspired by [sample_agents_azure_functions.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_azure_functions.py)
- [Getting Started with Azure Functions](https://learn.microsoft.com/azure/azure-functions/functions-get-started) page for more information on Azure Functions
- [Azure Blob storage bindings for Azure Functions overview](https://learn.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob?tabs=isolated-process%2Cextensionv5%2Cextensionv3&pivots=programming-language-python)

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

project_endpoint = os.environ["PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://mmai-swc-new01-prj-resource.services.ai.azure.com/api/projects/mmai-swc-new01-prj>
azure-ai-projects library installed version: 1.0.0b11
azure-ai-agents library installed version: 1.1.0b1


# Create AI Foundry Agents Client

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# Azure Storage Queues
The AI agent leverages Azure Functions triggered asynchronously via Azure Storage Queues. To enable the agent to perform Azure Function calls, you must set up the corresponding AzureFunctionTool, specifying input and output queues as well as parameter definitions.

# Azure Function Tool

In [3]:
from azure.ai.agents.models import AzureFunctionTool, AzureFunctionStorageQueue

os.environ["STORAGE_SERVICE_ENDPOINT"] = "https://mmaiswcnew01grpb306.queue.core.windows.net/"
storage_service_endpoint = os.environ["STORAGE_SERVICE_ENDPOINT"]
input_queue_name = "azure-function-foo-input"
output_queue_name = "azure-function-foo-output"

azure_function_tool = AzureFunctionTool(
    name="foo",
    description="Get answers from the foo bot.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "The question to ask."},
            "outputqueueuri": {"type": "string", "description": "The full output queue uri."},
        },
    },
    input_queue=AzureFunctionStorageQueue(
        queue_name=input_queue_name,
        storage_service_endpoint=storage_service_endpoint,
    ),
    output_queue=AzureFunctionStorageQueue(
        queue_name=output_queue_name,
        storage_service_endpoint=storage_service_endpoint,
    ),
)

print(f"azure_function_tool.definitions: {azure_function_tool.definitions}")
print(f"\nazure_function_tool.resources: {azure_function_tool.resources}")

azure_function_tool.definitions: [{'type': 'azure_function', 'azure_function': {'function': {'name': 'foo', 'description': 'Get answers from the foo bot.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The question to ask.'}, 'outputqueueuri': {'type': 'string', 'description': 'The full output queue uri.'}}}}, 'input_binding': {'storage_queue': {'queue_name': 'azure-function-foo-input', 'queue_service_endpoint': 'https://mmaiswcnew01grpb306.queue.core.windows.net/'}, 'type': 'storage_queue'}, 'output_binding': {'storage_queue': {'queue_name': 'azure-function-foo-output', 'queue_service_endpoint': 'https://mmaiswcnew01grpb306.queue.core.windows.net/'}, 'type': 'storage_queue'}}}]

azure_function_tool.resources: {}


In [5]:
function_name = "azure-function-04-queued-triggered"

agent = agents_client.create_agent(
    model=deployment_name,
    name=function_name,
    instructions=f"You are a helpful support agent. Use the provided function any time the prompt contains the string 'What would foo say?'. When you invoke the function, ALWAYS specify the output queue uri parameter as '{storage_service_endpoint}/azure-function-tool-output'. Always responds with \"Foo says\" and then the response from the tool.",
    tools=azure_function_tool.definitions,
)
print(f"Created agent, agent ID: {agent.id}")

HttpResponseError: (None) Invalid tool value(s): azure_function
Code: None
Message: Invalid tool value(s): azure_function

# Just for testing: invoke the Azure Function

In [ ]:
# Create agent with AI search tool and process assistant run
agent = agents_client.create_agent(
    model=deployment_name,
    name="azure-function-agent-foo",
    instructions="""
    You are a helpful support agent. Use the provided function any time the prompt contains the string 'What would foo say?'. 
    When you invoke the function, ALWAYS specify the output queue uri parameter as '{storage_service_endpoint}/azure-function-tool-output'. 
    Always responds with \"Foo says\" and then the response from the tool.
    """,
    tools=azure_function_tool.definitions,
    tool_resources=azure_function_tool.resources,
)

print(f"Created agent, agent ID: {agent.id}")

# Create the thread and attach a new message to it

In [ ]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What is the most prevalent element in the universe?" #  What would foo say?
)
print(f"Created message: {message}")

# Run the agent syncrhonously

In [ ]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Fetch messages from the thread after the agent run execution

In [ ]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

# Run Steps

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

# START teardown

In [ ]:
print(f"Deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

In [ ]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

# HIC SUNT LEONES